# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [315]:
!pip install -U scikit-learn

In [3]:
import pandas as pd

In [6]:
data = pd.read_csv('labeled.csv')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [8]:
data.toxic.value_counts(normalize=True)

,proportion
toxic,
0.0,0.66514
1.0,0.33486


In [9]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [10]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [11]:
vectorizer = CountVectorizer()
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

In [12]:
y = train.toxic.values
y_test = test.toxic.values

In [ ]:
clf = LogisticRegression(C=0.1, class_weight='balanced')

In [ ]:
# fit обучает модель
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [ ]:
# предикт предсказывает классы
preds = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       973
         1.0       0.73      0.83      0.77       469

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.85      1442



razdel.tokenize

In [170]:
!pip install razdel

In [171]:
from razdel import tokenize as razdel_tokenize

In [172]:
def tokenizer_razdel(text):
    return [token.text for token in razdel_tokenize(text)]

In [ ]:
vectorizer_razdel = CountVectorizer(tokenizer=tokenizer_razdel)
X_razdel = vectorizer_razdel.fit_transform(train.comment)
X_test_razdel = vectorizer_razdel.transform(test.comment)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
y_razdel = train.toxic.values
y_test_razdel = test.toxic.values

In [ ]:
clf_razdel = LogisticRegression(C=0.1, class_weight='balanced')

In [ ]:
# fit обучает модель
clf_razdel.fit(X_razdel, y_razdel)

LogisticRegression(C=0.1, class_weight='balanced')

In [ ]:
# предикт предсказывает классы
preds_razdel = clf_razdel.predict(X_test_razdel)

In [ ]:
print(classification_report(y_test_razdel, preds_razdel, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.85      0.88       973
         1.0       0.73      0.84      0.78       469

    accuracy                           0.85      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.86      0.85      0.85      1442



In [ ]:
print("Метрики для дефолтной токенизации:")
print(classification_report(y_test, preds, zero_division=0))
print("Метрики для токенизации razdel:")
print(classification_report(y_test_razdel, preds_razdel, zero_division=0))

Метрики для дефолтной токенизации:
              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       973
         1.0       0.73      0.83      0.77       469

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.85      1442

Метрики для токенизации razdel:
              precision    recall  f1-score   support

         0.0       0.92      0.85      0.88       973
         1.0       0.73      0.84      0.78       469

    accuracy                           0.85      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.86      0.85      0.85      1442



Векторайзер с токенизацией razdel немного лучше справился с задачей

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [14]:
!pip install nltk

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Decision Tree**

In [320]:
vectorizer_tree = CountVectorizer(stop_words=russian_stopwords,
                                  max_df=0.6,
                                  min_df=0.001,
                                  max_features=20000,
                                  ngram_range=(1,4))
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X_tree = vectorizer_tree.fit_transform(train.comment)
X_test_tree = vectorizer_tree.transform(test.comment)

y_tree = train.toxic.values
y_test_tree = test.toxic.values

clf_tree = DecisionTreeClassifier(max_depth=300,
                                  class_weight='balanced',
                                  min_samples_split=15,
                                  criterion='gini',
                                  min_samples_leaf=5)

In [321]:
# fit обучает модель
clf_tree.fit(X_tree, y_tree)

DecisionTreeClassifier(class_weight='balanced', max_depth=300,
                       min_samples_leaf=5, min_samples_split=15)

In [322]:
# предикт предсказывает классы
preds_tree = clf_tree.predict(X_test_tree)

In [323]:
print(classification_report(y_test_tree, preds_tree, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.72      0.78       957
         1.0       0.57      0.74      0.65       485

    accuracy                           0.73      1442
   macro avg       0.71      0.73      0.71      1442
weighted avg       0.75      0.73      0.73      1442



In [122]:
tree_probas = clf_tree.predict_proba(X_test_tree)[:, 1]

In [123]:
tree_probas

array([0.        , 0.67564366, 0.24885364, ..., 0.        , 0.02485118,
       0.67564366])

In [124]:
most_toxic_tree_indices = tree_probas.argsort()[-10:][::-1]

In [125]:
most_toxic_tree_indices

array([1142,  455,  841,  422,  438, 1092,  199,  193, 1354,  166])

In [126]:
most_toxic_tree = test.comment.iloc[most_toxic_tree_indices]

In [127]:
print("10 самых токсичных текстов для DecisionTreeClassifier:")
print(most_toxic_tree)

10 самых токсичных текстов для DecisionTreeClassifier:
1142    Блядь, ебаный ты шизоид, ты можешь нормально в...
455     Ну оль, ну вот че тебе надо? Иди в по сри, а э...
841     ссылки я давал, модератор в мягкой и корректно...
422     Да кремлеботов как урукхаев из грязи и говна д...
438     Н е сказал бы: понася могут ругать как пиздобо...
1092       Модератор не мой, я просто разместил объяву.\n
199     Модератор, может почистите скопившиеся репорты?\n
193     Мальчик сразу же ушел в глину по пояс Сука, эт...
1354                         Хорс пидор, тарков не дал!\n
166     на фестивале Панчлайн он конечно знатно обосра...
Name: comment, dtype: object


**Logistic Regression**

In [154]:
vectorizer_regression = CountVectorizer(stop_words=russian_stopwords,
                                  max_df=0.6,
                                  min_df=2,
                                  max_features=20000,
                                  ngram_range=(1,4))
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X_regression = vectorizer_regression.fit_transform(train.comment)
X_test_regression = vectorizer_regression.transform(test.comment)

y_regression = train.toxic.values
y_test_regression = test.toxic.values

clf_regression = LogisticRegression(C=0.4,
                                    class_weight='balanced',
                                    solver='saga',
                                    max_iter=200,
                                    tol=1e-4)

In [155]:
# fit обучает модель
clf_regression.fit(X_regression, y_regression)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.4, class_weight='balanced', max_iter=200, solver='saga')

In [156]:
# предикт предсказывает классы
preds_regression = clf_regression.predict(X_test_regression)

In [157]:
print(classification_report(y_test_regression, preds_regression, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       957
         1.0       0.72      0.83      0.77       485

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.82      1442
weighted avg       0.85      0.84      0.84      1442



In [158]:
regression_probas = clf_regression.predict_proba(X_test_regression)[:, 1]

In [160]:
most_toxic_regression_indices = regression_probas.argsort()[-10:][::-1]

In [162]:
most_toxic_regression = test.comment.iloc[most_toxic_regression_indices]

In [163]:
print("10 самых токсичных текстов для Logistic Regression:")
print(most_toxic_regression)

10 самых токсичных текстов для Logistic Regression:
1016    Раз уж вакханалия продолжается, давайте в этом...
965     АЛЛО БЛЯДЬ АБУ ТЫ Ч ОХУЕЛ? ВЫГОНЯЙ НАХУЙ СВОЕГ...
12      В очередной раз убеждаюсь, что двачеры редкост...
721     та ну, хуйня это все про джентельменство . мен...
1195    лахтадырые и ольгинцы (Лахта, Ольгино) это кот...
566     Вот это реально черт опущенный. Хоть Гомикадзе...
868     Пошла нахуй свинья. Хохла на штык! Хороший хох...
730     Я тебе дохуя пруфов привёл, а ты ебанный дебил...
1271    приходит умный человек и помогает ему научитьс...
1403    Какая убогая банда, гуляет на просторах YouTub...
Name: comment, dtype: object


**KNN**

In [305]:
vectorizer_KNN = TfidfVectorizer(stop_words=russian_stopwords,
                                 tokenizer=tokenizer_razdel,
                                 binary=True,
                                 ngram_range=(1, 1),
                                 max_df=0.9,
                                 min_df=3)
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X_KNN = vectorizer_KNN.fit_transform(train.comment)
X_test_KNN = vectorizer_KNN.transform(test.comment)

y_KNN = train.toxic.values
y_test_KNN = test.toxic.values

clf_KNN = KNeighborsClassifier(n_neighbors=30,
                               metric='cosine',
                               p=2,
                               n_jobs=-1,
                               weights='distance')

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [306]:
# fit обучает модель
clf_KNN.fit(X_KNN, y_KNN)

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=30,
                     weights='distance')

In [307]:
# предикт предсказывает классы
preds_KNN = clf_KNN.predict(X_test_KNN)

In [308]:
print(classification_report(y_test_KNN, preds_KNN, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.90      0.88       957
         1.0       0.79      0.70      0.74       485

    accuracy                           0.83      1442
   macro avg       0.82      0.80      0.81      1442
weighted avg       0.83      0.83      0.83      1442



In [309]:
KNN_probas = clf_KNN.predict_proba(X_test_KNN)[:, 1]

In [310]:
KNN_probas

array([0.23297793, 0.65334691, 0.46670677, ..., 0.1020813 , 0.        ,
       0.20024222])

In [311]:
most_toxic_KNN_indices = KNN_probas.argsort()[-10:][::-1]

In [312]:
most_toxic_KNN_indices

array([ 358, 1420,  375, 1129, 1387, 1153, 1162, 1181,   75,  463])

In [313]:
most_toxic_KNN = test.comment.iloc[most_toxic_KNN_indices]

In [314]:
print("10 самых токсичных текстов для KNeighborsClassifier:")
print(most_toxic_KNN)

10 самых токсичных текстов для KNeighborsClassifier:
358     Желательно гадить на быдло или попросту изолир...
1420    Этот расеянин уплатит за всё. Проблема не выби...
375                   Глянь DAWG(a) если ты анимеблядь.\n
1129    В России, слава богу, дураков ещё лет на сто п...
1387    продлил И правильно сделал, шизиков надо на це...
1153    Типа есть, чем думать. Лично мне жаль, что его...
1162         Валерианка Столичная тоже норм успокаивает\n
1181         Я свяжу тебе горло, и ты умрёшь. мимо айва\n
75      Анон, моя Родина - Татарстан - смертельно боль...
463     Весело ломать людям жизни ? В такие моменты я ...
Name: comment, dtype: object


Все модели выдали разные топ-10 токсичных текстов. KNN и Decision Tree ошиблись только в 1 случае из 10, остальные тексты действительно токсичные, не считая "1162         Валерианка Столичная тоже норм успокаивает\n" для KNN и "841     ссылки я давал, модератор в мягкой и корректно..." для Decision Tree. Почему валерьянка стала токсичной не понятно, но в примере с модератором содержится токсичное слово, на которое алгоритм и среагировал.

Логистическая регрессия выдала два неправильных текста, это :
1) 1016    Раз уж вакханалия продолжается, давайте в этом...
2) 1271    приходит умный человек и помогает ему научитьс...

В первом случае алгоритм среагировал на токсичный текст, который цитировался в комментарии, но сам по себе комментарий токсичным не был.
Во втором случае в комментарие тоже были токсичные слова и даже n-граммы, но контекст не был токсичным.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [588]:
import codecs

In [589]:
file = codecs.open('stopwords-ru.txt', 'r', 'utf_8_sig')
stopwords = file.read().split()
file.close()

In [332]:
import numpy as np

In [324]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [410]:
!pip install pymystem3

In [411]:
from pymystem3 import Mystem

In [412]:
mystem = Mystem(disambiguation=True)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [590]:
def lemmatizer(text):
  lemmas = mystem.lemmatize(text)
  lemmas = [lemma for lemma in lemmas if lemma.strip().isalpha()]
  lemmas = [lemma for lemma in lemmas if lemma not in stopwords]
  return lemmas

In [591]:
vectorizer_all = TfidfVectorizer(stop_words=russian_stopwords,
                                 tokenizer=lemmatizer,
                                 ngram_range=(1, 2),
                                 max_df=0.7,
                                 min_df=4)
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X_all = vectorizer_all.fit_transform(train.comment)
X_test_all = vectorizer_all.transform(test.comment)

y_all = train.toxic.values
y_test_all = test.toxic.values

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


**Logistic Regression**

In [592]:
clf_logreg = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', class_weight='balanced')

In [593]:
# fit обучает модель
clf_logreg.fit(X_all, y_all)

LogisticRegression(class_weight='balanced', solver='liblinear')

In [594]:
# предикт предсказывает классы
preds_logreg = clf_logreg.predict(X_test_all)

In [595]:
print(classification_report(y_test_all, preds_logreg, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89       957
         1.0       0.77      0.82      0.80       485

    accuracy                           0.86      1442
   macro avg       0.84      0.85      0.84      1442
weighted avg       0.86      0.86      0.86      1442



In [596]:
# Получение весов и признаков
coef_logreg = clf_logreg.coef_[0]  # Веса признаков для токсичного класса
feature_names_logreg = vectorizer_all.get_feature_names_out()

In [597]:
# Сортировка по модулю весов
sorted_indices_logreg = np.argsort(np.abs(coef_logreg))[-5:]

In [598]:
print("Топ-5 токсичных слов для Logistic Regression:")
for idx in reversed(sorted_indices_logreg):
    print(feature_names_logreg[idx], round(coef_logreg[idx], 4))

Топ-5 токсичных слов для Logistic Regression:
хохол 5.3557
тупой 4.4546
дебил 4.1712
хохлов 4.1462
шлюха 3.7675


**Decision Trees**

In [599]:
clf_decision_tree = DecisionTreeClassifier(max_depth=300,
                                  class_weight='balanced',
                                  min_samples_split=20,
                                  criterion='gini',
                                  min_samples_leaf=3,
                                  splitter='random')

In [600]:
# fit обучает модель
clf_decision_tree.fit(X_all, y_all)

DecisionTreeClassifier(class_weight='balanced', max_depth=300,
                       min_samples_leaf=3, min_samples_split=20,
                       splitter='random')

In [601]:
# предикт предсказывает классы
preds_decision_tree = clf_decision_tree.predict(X_test_all)

In [602]:
print(classification_report(y_test_all, preds_decision_tree, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.80      0.94      0.87       957
         1.0       0.82      0.55      0.66       485

    accuracy                           0.81      1442
   macro avg       0.81      0.74      0.76      1442
weighted avg       0.81      0.81      0.80      1442



In [603]:
# Получение важностей и признаков
importances_decision_tree = clf_decision_tree.feature_importances_  # Важности признаков
feature_names_decision_tree = vectorizer_all.get_feature_names_out()

# Сортировка по важностям
sorted_indices_decision_tree = np.argsort(importances_decision_tree)[-5:]

print("Топ-5 токсичных слов для Decision Tree:")
for idx in reversed(sorted_indices_decision_tree):
    print(feature_names_decision_tree[idx], round(importances_decision_tree[idx], 4))

Топ-5 токсичных слов для Decision Tree:
хохол 0.0367
тупой 0.019
ебать 0.0186
нахуй 0.0155
дебил 0.0152


**Naive Bayes**

In [614]:
clf_naive_bayes = MultinomialNB(alpha=1.0, class_prior=[0.7, 0.3], fit_prior=False)

In [615]:
# fit обучает модель
clf_naive_bayes.fit(X_all, y_all)

MultinomialNB(class_prior=[0.7, 0.3], fit_prior=False)

In [616]:
# предикт предсказывает классы
preds_naive_bayes = clf_naive_bayes.predict(X_test_all)

In [617]:
print(classification_report(y_test_all, preds_naive_bayes, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.97      0.90       957
         1.0       0.92      0.65      0.76       485

    accuracy                           0.86      1442
   macro avg       0.88      0.81      0.83      1442
weighted avg       0.87      0.86      0.85      1442



In [618]:
# Получение логарифмов вероятностей и признаков
log_prob_naive_bayes = clf_naive_bayes.feature_log_prob_[1]  # Логарифмы вероятностей для токсичного класса
feature_names_naive_bayes = vectorizer_all.get_feature_names_out()

# Сортировка по вероятностям
sorted_indices_naive_bayes = np.argsort(log_prob_naive_bayes)[-5:]

print("Топ-5 токсичных слов для Naive Bayes:")
for idx in reversed(sorted_indices_naive_bayes):
    print(feature_names_naive_bayes[idx], round(log_prob_naive_bayes[idx], 4))

Топ-5 токсичных слов для Naive Bayes:
хохол -5.5347
тупой -6.0699
хохлов -6.1214
ебать -6.1715
нахуй -6.1982


**RandomForest**

In [634]:
clf_random_forest = RandomForestClassifier(n_estimators=200,
                                           max_depth=30,
                                           min_samples_split=3,
                                           min_samples_leaf=2,
                                           class_weight='balanced')

In [635]:
# fit обучает модель
clf_random_forest.fit(X_all, y_all)

RandomForestClassifier(class_weight='balanced', max_depth=30,
                       min_samples_leaf=2, min_samples_split=3,
                       n_estimators=200)

In [636]:
# предикт предсказывает классы
preds_random_forest = clf_random_forest.predict(X_test_all)

In [637]:
print(classification_report(y_test_all, preds_random_forest, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88       957
         1.0       0.83      0.61      0.71       485

    accuracy                           0.83      1442
   macro avg       0.83      0.77      0.79      1442
weighted avg       0.83      0.83      0.82      1442



In [638]:
# Получение важностей и признаков
importances_random_forest = clf_random_forest.feature_importances_  # Важности признаков
feature_names_random_forest = vectorizer_all.get_feature_names_out()

# Сортировка по важностям
sorted_indices_random_forest = np.argsort(importances_random_forest)[-5:]

print("Топ-5 токсичных слов для Random Forest:")
for idx in reversed(sorted_indices_random_forest):
    print(feature_names_random_forest[idx], round(importances_random_forest[idx], 4))

Топ-5 токсичных слов для Random Forest:
хохол 0.0377
хохлов 0.0229
тупой 0.0226
ебать 0.0185
дебил 0.0184
